동일한 이미지를 검색할 때는 MD5와 SHA256 등의 해쉬 함수를 사용하면 되지만 이 해쉬 함수로는 비슷하 이미지는 검색할 수 없다.

이미지가 조금 다르더라도 유사한지를 검출해야 할 때는 Average Hash를 사용합니다.

1. 이미지 크기를 8x8로 축소합니다.
2. 색을 그레이스케일로 변환합니다.
3. 이미지의 각 픽셀의 평균을 계산합니다.
4. 각 픽셀의 어두운 정도가 평균보다 크면 1 평균보다 작음ㄴ 0ㅇ으로 입력합니다.

위와 같은 간단한 방법으로 이미지의 형태를 나타내는 64비트 해시 값을 구할 수 있습니다.

pip3 install Pillow

In [1]:
from PIL import Image
import numpy as np

In [4]:
def average_hash(fname, size = 16):
    img = Image.open(fname)
    img = img.convert('L')
    img = img.resize((size, size), Image.ANTIALIAS)
    pixel_data = img.getdata()
    pixels = np.array(pixel_data)
    pixels = pixels.reshape((size, size))
    avg = pixels.mean()
    diff = 1 * (pixels > avg)
    return diff

def np2hash(ahash):
    bhash = []
    for nl in ahash.tolist():
        s1 = [str(i) for i in nl]
        s2 = "".join(s1)
        i = int(s2, 2) #이진수를 정수로 변환하기
        bhash.append('%04x' % i)
    return "".join(bhash)

ahash = average_hash('tower.jpg')
print(ahash)
print(np2hash(ahash))
    

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0]
 [1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0]
 [1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]]
000001000180018003c003c003c087e07ff05ff07ff2cffa9ffe3ffe3ffe1fff


한 행의 숫자 요소를 16진수로 바꿔서 이어 붙임

http://www.vision.caltech.edu/Image_Datasets/Caltech101/Caltech101.html


In [13]:
from PIL import Image
import numpy as np
import os, re

search_dir = '../data/image/101_ObjectCategories'
cache_dir = '../data/image/cache_avhash'

if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)
    
    
def average_hash(fname, size = 16):
    fname2 = fname[len(search_dir):]
    
    cache_file = cache_dir + '/' + fname2.replace('/', '_') + '.csv'
   # print(cache_file)
    if not os.path.exists(cache_file):
        img = Image.open(fname)
        img = img.convert('L').resize((size, size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size, size))
        avg = pixels.mean()
        px = 1 * (pixels > avg)
        np.savetxt(cache_file, px, fmt='%.0f', delimiter=',')
    else:
        px = np.loadtxt(cache_file, delimiter=',')
    return px

def hamming_dist(a, b):
    aa = a.reshape(1,-1)
    ab = b.reshape(1,-1)
    dist = (aa != ab).sum()
    return dist

def enum_all_files(path):
    for root, disrs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root,f)
            print(fname)
            if re.search(r'\.(jpg|jpeg|png)$', fname):
                yield fname
                
def find_image(fname, rate):
    src = average_hash(fname)
    for fname in enum_all_files(search_dir):
        dst = average_hash(fname)
        diff_r = hamming_dist(src, dst) / 256
        if diff_r < rate:
            yield (diff_r, fname)
            
srcfile = search_dir + '/chair/image_0016.jpg'            
html = ""
sim =list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x:x[0])
for r, f in sim:
    print(r, ">", f)
    s = '<div style="float:left;"><h3>[ 차이 :' + str(r) + '-' + \
        os.path.basename(f) + ']</h3>'+ \
        '<p><a href="' + f + '"><img src="' + f + '" width=400>'+ \
        '</a></p></div>'
    html += s
# HTML로 출력하기
html = """<html><head><meta charset="utf8"></head>
<body><h3>원래 이미지</h3><p>
<img src='{0}' width=400></p>{1}
</body></html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:
    f.write(html)
print("ok")

../data/image/101_ObjectCategories\accordion\image_0001.jpg


FileNotFoundError: [Errno 2] No such file or directory: '../data/image/cache_avhash/\\accordion\\image_0001.jpg.csv'

In [8]:
a = np.random.normal(1,2,(3,3))
np.savetxt('savetext.txt', a, fmt='%.2f', delimiter=',')